In [ ]:
%load_ext autoreload
%autoreload 2
%aimport

In [ ]:
%matplotlib inline 

In [ ]:
import matplotlib.pyplot as plt
import torch

from bliss.inference import SDSSFrame
from bliss.datasets import sdss
from bliss.inference import reconstruct_scene_at_coordinates
from case_studies.sdss_galaxies.plots.main import load_models


from astropy.table import Table

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# check GPU is configured correctly
device = torch.device('cuda:0')
!echo $CUDA_VISIBLE_DEVICES

# Load data

In [ ]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../../config"):
    cfg = compose("config", overrides=[])
    
    
enc, dec = load_models(cfg, device)
bp = enc.border_padding
torch.cuda.empty_cache()

In [ ]:
#ra_lim = (336.5825, 336.7461)
#dec_lim = (-1.044, -0.8189)

bp = 24
frame = instantiate(cfg.plots.frame, cat_type='decals', cat_file="../../../../data/sweep-330m005-340p000.fits")
h_topleft = bp
w_topleft = bp
h_end = ((frame.image.shape[2] - 2 * bp) // 4) * 4 + bp
w_end = ((frame.image.shape[3] - 2 * bp) // 4) * 4 + bp
hlim = (h_topleft, h_end)
wlim = (w_topleft, w_end)

decals_cat = frame.get_catalog(hlim, wlim)
decals = decals_cat.apply_mag_bin(-np.inf, 23)
decals.n_sources

# Get Locations

In [ ]:
#inference
with torch.no_grad():
    _, tile_est = reconstruct_scene_at_coordinates(
        enc,
        dec,
        frame.image,
        frame.background,
        h_range=hlim,
        w_range=wlim,
    )

In [ ]:
tile_est.set_all_fluxes_and_mags(dec)
map_recon = tile_est.cpu().to_full_params()
torch.cuda.empty_cache()

In [ ]:
# prepare inference locs
plocs = map_recon.plocs.cpu().numpy().squeeze() + bp - 0.5 # plotting adjustment
coords = frame.wcs.all_pix2world(np.hstack([plocs[:, 1, None], plocs[:, 0, None]]), 0)
galaxy_bool = map_recon['galaxy_bools'].numpy().astype(bool).squeeze()
galaxy_prob = map_recon['galaxy_probs'].numpy().squeeze()
mags = map_recon['mags'].squeeze().numpy()
n_probs = np.exp(map_recon['n_source_log_probs'].squeeze().numpy())

In [ ]:
# coadd params
coplocs = decals.plocs.squeeze().numpy() + bp - 0.5 # plotting adjustment
cogbool = decals['galaxy_bools'].squeeze().numpy().astype(bool)
cobjid = decals['objid'].squeeze().numpy()

cora = decals['ra'].squeeze().numpy()
codec = decals['dec'].squeeze().numpy()
comags = decals['mags'].squeeze().numpy()

# Plotly Frame

In [ ]:
# cut on mag
lim_mag = 20 
keep = mags < 20
plocs = plocs[keep]
coords = coords[keep]
galaxy_bool = galaxy_bool[keep]
galaxy_prob = galaxy_prob[keep]
n_probs = n_probs[keep]
mags = mags[keep]

cokeep = comags < 23
coplocs = coplocs[cokeep]
cogbool = cogbool[cokeep]
cora = cora[cokeep]
codec = codec[cokeep]
comags = comags[cokeep]
cobjid = cobjid[cokeep]

In [ ]:
from bliss.reporting import match_by_locs

coindx, indx, dkeep, _ = match_by_locs(torch.from_numpy(coplocs), torch.from_numpy(plocs))
nindx = np.array(list(set(range(len(plocs))).difference(set(indx)))).astype(int)
assert len(indx) + len(nindx) == len(plocs)

gal_misclass = ~np.equal(galaxy_bool[indx][dkeep], cogbool[coindx][dkeep]) & cogbool[coindx][dkeep].astype(bool)
plocs_misclassified = plocs[indx][dkeep][gal_misclass]
plocs_unmatched = plocs[nindx]
plocs_unmatched_dist = plocs[indx][~dkeep]

In [ ]:

image = frame.image.squeeze().numpy()

fig = px.imshow(image, width=800, height=550, zmin=800, zmax=1150, color_continuous_scale='gray')


# create scatter objects
scatter_coadd_galaxies = go.Scatter(name='galaxy coadd', x=coplocs[cogbool][:, 1], y=coplocs[cogbool][:, 0], 
                                  mode='markers', marker=dict(color='red', size=8, symbol='cross'), 
                                 hovertext=[f'mag:{x:.2f}; (ra, dec):({y:.4f}, {z:.4f}); objid: {w:.1f}' 
                                            for (x, y, z, w) in zip(comags[cogbool], 
                                                             cora[cogbool],
                                                             codec[cogbool], cobjid[cogbool])]
                                 )
scatter_coadd_star = go.Scatter(name='star coadd', x=coplocs[~cogbool][:, 1], y=coplocs[~cogbool][:, 0], 
                                  mode='markers', marker=dict(color='blue', size=8, symbol='cross'),
                                 hovertext=[f'mag:{x:.2f}; (ra, dec):({y:.4f}, {z:.4f}); objid: {w:.1f}' 
                                            for (x, y,z,w) in zip(comags[~cogbool], 
                                                             cora[~cogbool],
                                                             codec[~cogbool],cobjid[~cogbool])]
                                 )
scatter_bliss_galaxies = go.Scatter(name='galaxy pred', x=plocs[galaxy_bool][:, 1], y=plocs[galaxy_bool][:, 0], 
                                  mode='markers', marker=dict(color='hotpink', size=11, symbol='x'),
                                 hovertext=[f'mag:{a:.2f}; prob_galaxy:{x:.2f}; (ra, dec):({y:.4f}, {z:.4f}); n_prob: {w:.4f}' 
                                            for (a, x, y,z,w) in zip(mags[galaxy_bool], 
                                                               galaxy_prob[galaxy_bool], 
                                                               coords[galaxy_bool][:, 0], 
                                                               coords[galaxy_bool][:, 1], n_probs[galaxy_bool])]
                                 )
scatter_bliss_star = go.Scatter(name='star pred', x=plocs[~galaxy_bool][:, 1], y=plocs[~galaxy_bool][:, 0], 
                                mode='markers', marker=dict(color='cyan', size=11, symbol='x'),\
                                 hovertext=[f'mag:{a:.2f}; prob_galaxy:{x:.2f}; (ra, dec):({y:.4f},{z:.4f}); n_prob:{w:.4f}' 
                                            for (a, x,y,z,w) in zip(mags[~galaxy_bool], 
                                                               galaxy_prob[~galaxy_bool], 
                                                               coords[~galaxy_bool][:, 0], 
                                                               coords[~galaxy_bool][:, 1], n_probs[~galaxy_bool])]
                                 )

scatter_unmatched = go.Scatter(name='unmatched', x=plocs_unmatched[:, 1], y=plocs_unmatched[:, 0], 
                                mode='markers', marker=dict(color='magenta', size=10, symbol='cross'),
                                 )
scatter_unmatched_dist = go.Scatter(name='unmatched (distance)', x=plocs_unmatched_dist[:, 1], y=plocs_unmatched_dist[:, 0], 
                                mode='markers', marker=dict(color='green', size=10, symbol='cross'),
                                 )
scatter_misclassified = go.Scatter(name='misclassified', x=plocs_misclassified[:, 1], y=plocs_misclassified[:, 0], 
                                mode='markers', marker=dict(color='yellow', size=10, symbol='cross'),
                                 )

# add traces to figure
fig.add_trace(scatter_coadd_galaxies)
fig.add_trace(scatter_coadd_star)
fig.add_trace(scatter_bliss_galaxies)
fig.add_trace(scatter_bliss_star)
fig.add_trace(scatter_unmatched)
fig.add_trace(scatter_unmatched_dist)
fig.add_trace(scatter_misclassified)


fig.update_layout(legend=dict(orientation="h", y=1.05)) # adjust legend


fig.show()